# 6. Tuning

In [1]:
MODEL = 'DNNc'
TUNING_MODEL = 'guangan'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
  import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [5]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [6]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [7]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [8]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [9]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [10]:
import GuangAn_H2H

In [11]:
guangan = GuangAn_H2H.GuangAn()

GuangAn Initialised


In [12]:
from models.DenseNeuralNetwork import DNNR_const_sk as DNN

In [13]:
guangan.read_in_model(DNN, 'Regression')

Successfully read in model <class 'models.DenseNeuralNetwork.DNNR_const_sk'>, which is a Regression model


In [14]:
guangan.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [15]:
parameter_choices = {
    'n_hidden_layers': {1, 2, 3, 4, 5},
    'alpha': (-8, -1),
    'learning_rate_init': (-4, -1),
    'activation': ('relu', 'logistic'),
    'hidden_layer_n_neurons': set(range(1, 61))
}

In [16]:
guangan.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [17]:
guangan.read_in_transform({'learning_rate_init': '10^', 'alpha': '10^'})

Updated transform dictionary: {'n_hidden_layers': False, 'alpha': '10^', 'learning_rate_init': '10^', 'activation': False, 'hidden_layer_n_neurons': False}


In [18]:
guangan.read_in_categorical(['activation'])

Updated categorical dictionary: {'n_hidden_layers': False, 'alpha': False, 'learning_rate_init': False, 'activation': True, 'hidden_layer_n_neurons': False}
Updated original bounds dict: [({1, 2, 3, 4, 5}, 'n_hidden_layers'), ((-8, -1), 'alpha'), ((-4, -1), 'learning_rate_init'), ({'values': ('logistic', 'relu')}, 'activation'), ({1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60}, 'hidden_layer_n_neurons')]


In [19]:
guangan.set_non_tuneable_hyperparameters({'random_state': 19260817, 'max_iter': 1000, 'learning_rate': 'adaptive', 'batch_size': 'auto'})

Successfully recorded non_tuneable_hyperparameter choices


In [20]:
guangan.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [21]:
try:
  guangan.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv', f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')
  print('True')
except:
  print('False')

Successfully read in tuning result of 1911 rows
True


In [22]:
guangan.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [23]:
guangan.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [24]:
guangan.train_other_info = train_other_info
guangan.val_other_info = val_other_info
guangan.test_other_info = test_other_info

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
guangan.tune()

Streaming output truncated to the last 5000 lines.
        Current best combo: (3, -4.5, -2.5, 'logistic', 60, 35) with val score 0.6195. 
        Has trained 1911 combinations so far
Already Trained and Tested combination {'n_hidden_layers': 2, 'alpha': -2.3125, 'learning_rate_init': -3.90625, 'activation': 'relu', 'hidden_layer_n_neurons': 18, 'features': 11}, which had val score of 0.5671
        Current best combo: (3, -4.5, -2.5, 'logistic', 60, 35) with val score 0.6195. 
        Has trained 1911 combinations so far
Already Trained and Tested combination {'n_hidden_layers': 2, 'alpha': -2.3125, 'learning_rate_init': -3.90625, 'activation': 'relu', 'hidden_layer_n_neurons': 18, 'features': 12}, which had val score of 0.5681
        Current best combo: (3, -4.5, -2.5, 'logistic', 60, 35) with val score 0.6195. 
        Has trained 1911 combinations so far
Already Trained and Tested combination {'n_hidden_layers': 2, 'alpha': -2.3125, 'learning_rate_init': -3.90625, 'activation': 'r

In [28]:
guangan.view_best_combo_and_score()

Max Score: 
 0.6194968553459119
Max Test Score: 
 0.619497
Max Combo Index: 
 (3, -4.5, -2.5, 'logistic', 60, 35)
Max Combo Hyperparamer Combination: 
 {'n_hidden_layers': 3, 'alpha': -4.5, 'learning_rate_init': -2.5, 'activation': 'logistic', 'hidden_layer_n_neurons': 60, 'features': 35}
Max Combo Features: 
 ('AFLCA_votes', 'Disposals', 'Contested Possessions', 'Goals', 'Centre Clearances', 'Intercepts', 'Clearances', 'Marks Inside 50', 'Hitouts', 'Handballs', 'Behinds', 'Contested Marks', 'Bounces', 'Ineffective Disposals', 'Stoppage Clearances', 'Tackles Inside 50', 'Rebound 50s', 'One Percenters', 'Goal Assists', 'Uncontested Possessions', 'Frees Agains', 'Inside 50s', 'Frees For', 'Metres Gained', 'Score Involvements', 'Behind Assists', 'Tackles Outside 50', 'Tackles', 'Clangers', 'Uncontested Marks', 'Marks Outside 50', 'Kicks', 'Turnovers', 'Marks', 'Time On Ground %', 'Effective Disposals', 'HomeAway', 'Winloss')
# Combos Checked: 2012
